#### What is vectorization?

NumPy vectorization involves performing mathematical operations on `entire arrays`, eliminating the need to loop through individual elements, which is notoriously slow in Python

The aspect that is relevant to us for now is `broadcasting`

#### Broadcasting

(from James Murphy https://mcoding.io/)

The term broadcasting describes how NumPy treats `arrays with different shapes` during operations in a practical and reasonable way, providing a means of `vectorizing` array operations

So, what do we mean by practical and reasonable?

For example

$$\begin{bmatrix}0 & 1 & 2 \\
3 & 4 & 5\end{bmatrix}+\begin{bmatrix}1 & 2 & 3\end{bmatrix}=\begin{bmatrix}1 & 3 & 5 \\ 4 & 6 & 8\end{bmatrix}$$

and

$$\begin{bmatrix}0 & 1 & 2 \\
3 & 4 & 5\end{bmatrix}+\begin{bmatrix}1 \\ 2\end{bmatrix}=\begin{bmatrix}1 & 2 & 3 \\ 5 & 6 & 7\end{bmatrix}$$

It is clear how these operations would typically be handled, even though the matrices have different shapes

Broadcasting aims to fulfill such operations, and it works with plus, minus, times, exponential, min/max and more operations we can think of

#### Broadcasting rules

The rules of broadcasting are

* If two arrays have `different number of dimensions`, prepend `1`'s to the shape of the array with smaller number of dimensions
* After the number of dimensions matches, any dimension for which one array has a `size of 1` can be broadcasted to the `size of this dimension in the other array`
* All other dimensions must have matching sizes

We can verify that these rules are used in the two examples above

In the first example, the shape of the arrays are (2, 3) and (1, 3), then using the broadcasting rules

* we see the first dimension can be broadcasted as the size of this dimension for the second array is `1`
* after broadcasting to the size of the first dimension of the first array, which is `2`, the broadcasted version of the second array is
$$\begin{bmatrix}1 & 2 & 3 \\ 1 & 2 & 3\end{bmatrix}$$
which has shape (2, 3), making it compatible with the first array

Similarly, in the second example, the shape of the arrays are (2, 3) and (2, 1), then using the broadcasting rules

* we see the second dimension can be broadcasted as the size of this dimension for the second array is `1`
* after broadcasting to the size of the second dimension of the first array, which is `3`, the broadcasted version of the second array is
$$\begin{bmatrix}1 & 1 & 1 \\ 2 & 2 & 2\end{bmatrix}$$
which has shape (2, 3), making it compatible with the first array

#### A useful case of broadcasting

A useful case for our purpose is to compute the kernel matrix $X_k$, for example, for an array $x=x_1 ,\cdots, x_n$, we want each entry of $X_{k}$ to be

$$X_{k, ij}=\exp\left(-\frac{1}{2}(x_i-x_j)^2\right)$$

In [6]:
import numpy as np
import time
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

In [7]:
x = (np.linspace(0, 1, 1000)).reshape(-1, 1)
# This is what our data would look like
# dim[0] is number of samples, and dim[1] is number of features
print(x.shape)

(1000, 1)


In [8]:
def rbf_kernel(x_set_1, x_set_2):
    if np.ndim(x_set_2) == 1:
        x_set_2 = x_set_2.reshape(-1, 1)
    if np.ndim(x_set_1) == 1:
        x_set_1 = x_set_1.reshape(-1, 1)
    num_rows = x_set_1.shape[0]
    num_cols = x_set_2.shape[0]
    k_mat = np.zeros((num_rows, num_cols))
    for i in range(num_rows):
        for j in range(num_cols):
            sq_dist = np.sum((x_set_1[i,:] - x_set_2[j,:])**2)
            k_mat[i, j] = np.exp(-sq_dist / 2)

    return k_mat

In [9]:
def rbf_kernel_vectorized(x_set_1, x_set_2):
    num_points_1 = x_set_1.shape[0]
    num_points_2 = x_set_2.shape[0]

    x1_sq_norms = np.sum(x_set_1 ** 2, axis=1).reshape(num_points_1, 1)  # Shape (num_points_1, 1)
    x2_sq_norms = np.sum(x_set_2 ** 2, axis=1).reshape(1, num_points_2)  # Shape (1, num_points_2)

    # Broadcasting step
    sq_dists = x1_sq_norms + x2_sq_norms - 2 * np.dot(x_set_1, x_set_2.T)  # Shape (num_points_1, num_points_2)
    k_mat = np.exp(-sq_dists / 2)

    return k_mat

In [10]:
start_time = time.time()
rbf_kernel = rbf_kernel(x, x)
print(rbf_kernel.shape)
print(f"Time taken using loop: {time.time() - start_time:.4f} seconds")

start_time = time.time()
rbf_kernel_vectorized = rbf_kernel_vectorized(x, x)
print(rbf_kernel_vectorized.shape)
print(f"Time taken using broadcasting: {time.time() - start_time:.4f} seconds")

(1000, 1000)
Time taken using loop: 9.6198 seconds
(1000, 1000)
Time taken using broadcasting: 0.0334 seconds
